__Import libraries__

In [1]:
from sense_hat import SenseHat
import time
import datetime
from collections import OrderedDict

__Define Colours__

In [2]:
n = [0,0,0]
g = [0,255,0]
b = [0,0,255]
o = [255,127,0]

__Define Shapes__

In [3]:
fhg_shape = [
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,g,g,g,g,
b,b,g,g,g,o,o,g,
n,n,n,n,g,g,g,g,
n,n,n,n,n,n,n,n
]

fhv_shape = [
n,n,n,n,n,n,n,n,
n,n,n,n,g,g,g,g,
b,b,g,g,g,o,o,g,
n,n,n,n,g,g,g,g,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n
]

bhg_shape = [
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
g,g,g,g,n,n,n,n,
g,o,o,g,g,g,b,b,
g,g,g,g,n,n,n,n,
n,n,n,n,n,n,n,n
]

bhv_shape = [
n,n,n,n,n,n,n,n,
g,g,g,g,n,n,n,n,
g,o,o,g,g,g,b,b,
g,g,g,g,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,n,n,n,n
]

ser1_shape = [
n,n,n,n,g,g,g,n,
n,n,n,n,g,o,g,n,
n,n,n,n,g,o,g,n,
n,n,n,n,g,g,g,n,
n,n,n,n,n,g,n,n,
n,n,n,n,n,g,n,n,
n,n,n,n,n,b,n,n,
n,n,n,n,n,b,n,n
]

ser2_shape = [
g,g,g,n,n,g,g,g,
g,o,g,n,n,g,o,g,
g,o,g,n,n,g,o,g,
g,g,g,n,n,g,g,g,
n,g,n,n,n,n,g,n,
n,g,n,n,n,n,g,n,
n,b,n,n,n,n,b,n,
n,b,n,n,n,n,b,n
]

sma_shape = [
n,n,g,g,g,g,n,n,
n,g,o,o,o,o,g,n,
n,g,o,o,o,o,g,n,
n,n,g,g,g,g,n,n,
n,n,n,g,g,n,n,n,
n,n,n,g,g,n,n,n,
n,n,n,b,b,n,n,n,
n,n,n,b,b,n,n,n
]

qm_shape = [
n,n,n,o,o,o,n,n,
n,n,o,n,n,n,o,n,
n,n,o,n,n,n,o,n,
n,n,n,n,n,o,n,n,
n,n,n,n,o,n,n,n,
n,n,n,n,o,n,n,n,
n,n,n,n,n,n,n,n,
n,n,n,n,o,n,n,n
]

all_shapes = OrderedDict([("fhg", fhg_shape),
                          ("fhv", fhv_shape),
                          ("bhg", bhg_shape),
                          ("bhv", bhv_shape),
                          ("sv1", ser1_shape),
                          ("sv2", ser2_shape),
                          ("sma", sma_shape)])

__Initialise Objects and Variables__

In [4]:
sense = SenseHat()
sense.set_imu_config(False, True, True) # Switch off Magnenometer
sense.low_light = True
sense.set_pixels(qm_shape) # Display question mark

shape_pos = -1 # Indexes in Python start at 0 not 1
choice_mode = True
log_mode = False
sampling_rate = 1.0/10.0 # 0.1 second most accurate

__Log Data__

In [5]:
while choice_mode:
    # Wait for button push
    for event in sense.stick.get_events():
        # Only look at released not pressed
        if event.action == "released":
            if event.direction == "right":
                # Wraps around the shots
                shape_pos = (shape_pos + 1) % len(all_shapes)
            if event.direction == "left":
                shape_pos = (shape_pos - 1) % len(all_shapes)
            sense.set_pixels(all_shapes[all_shapes.keys()[shape_pos]])
            
            if event.direction == "middle" and shape_pos >= 0:
                choice_mode = False
                log_mode = True
                records = "t,GyrX,GyrY,GyrZ,AccX,AccY,AccZ\n"
                # Display negative of image
                sense.set_pixels(map(lambda x: [abs(y - 175) for y in x], all_shapes[all_shapes.keys()[shape_pos]]))
                save_pos = shape_pos
                shape_pos = -1
                # Enter loggin mode
                while log_mode:
                    # Get a time stamp
                    time_stamp = time.time()
                    # Read and save IMU values
                    gyro = sense.get_gyroscope_raw()
                    acceleration = sense.get_accelerometer_raw()
                    
                    records = records + datetime.datetime.fromtimestamp(time.time()).strftime('%H:%M:%S.%f') + ","
                    records = records + "{0:.5f},{1:.5f},{2:.5f},".format(gyro['x'],gyro['y'],gyro['z'])
                    records = records + "{0:.5f},{1:.5f},{2:.5f}".format(acceleration['x'],acceleration['y'],acceleration['z'])
                    records = records + "\n"
                    
                    # Wait for the sampling period
                    # Don't use sleep() as the OS takes over and will delay giving back the process priority
                    while time.time() <= time_stamp + sampling_rate:
                        pass
                    
                    for event in sense.stick.get_events():
                        if event.action == "released":
                            if event.direction == "middle":
                                # Save file with recorded values
                                file_name = all_shapes.keys()[save_pos]
                                file_name = file_name + "_"
                                file_name = file_name + datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d_%H:%M:%S')
                                file_name = file_name + ".csv"
                                f = open(file_name,"a+")
                                f.write(records)
                                f.close()
                                choice_mode = True
                                log_mode = False
                                # Display question mark
                                sense.set_pixels(qm_shape)
            # Come out of choice mode gracefully
            elif event.direction == "middle" and shape_pos < 0:
                choice_mode = False
                sense.clear()

In [6]:
sense.clear()